In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
from tqdm import tqdm

import sys

tf.config.set_visible_devices([], 'GPU')

sys.path.append('../src/')

from mlp import datasets as mlp_datasets
from mlp import models as mlp_models

from utils import metrics


In [2]:
def generate_output(model_obj,
                    model_params,
                    model_weights,
                    save_path):


    if not os.path.isdir(save_path):
        os.makedirs(save_path)
    
    print(best_params)
    
    batch_size = model_params['batch_size']
    num_epochs = model_params['num_epochs']
    bits = model_params['bits']
    radius = model_params['radius']
    use_counts = model_params['use_counts']

    del model_params['batch_size']
    del model_params['num_epochs']
    del model_params['bits']
    del model_params['radius']
    del model_params['use_counts']
    
    train, valid, test_1, test_2 = mlp_datasets.get_ecfp_datasets(
        f"../input/datasets/{save_path.split('/')[-2]}.csv",
        bits=bits, radius=radius, use_counts=use_counts,
    )
    
    for name, dataset in zip(['train', 'valid', 'test_1', 'test_2'], [train, valid, test_1, test_2]):
        
        if dataset is not None:
            model = model_obj(**model_params)

            model(train['X'][:1])
            model.set_weights(model_weights)
            
            y_pred = model.predict(dataset['X'], dataset['y'])[1]

            np.save(save_path + '/' + name, np.stack([dataset['y'], y_pred]))
    

In [3]:
parameters = {
    'num_layers':    lambda:       np.random.randint(*[1, 3+1]),
    'num_units':     lambda:       np.random.randint(*[256, 1024+1]),
    'learning_rate': lambda:       np.random.uniform(*[1e-4, 1e-3]),
    'num_epochs':    lambda:       np.random.randint(*[50, 200+1]),
    'batch_size':    lambda:       np.random.choice([32, 64, 128]),
    'dropout_rate':  lambda:       np.random.uniform(*[0.0, 0.3]),
    'bits':          lambda:       np.random.randint(*[512, 2048+1]),
    'radius':        lambda:       np.random.randint(*[1, 3+1]),
    'use_counts':    lambda:       np.random.choice([True, False]),
}

model_name = 'mlp'

dataset_names = ['RIKEN', 'Fiehn_HILIC', 'SMRT'] # list(configuration.datasets.keys())

NUM_SEARCHES = 20

In [ ]:
for dataset_name in dataset_names:
    
    best_error = float('inf')
    
    for i in range(NUM_SEARCHES):
        
        np.random.seed(42+i)
        
        num_layers = parameters['num_layers']()
        num_units = parameters['num_units']()
        learning_rate = parameters['learning_rate']()
        batch_size = parameters['batch_size']()
        num_epochs = parameters['num_epochs']()
        dropout_rate = parameters['dropout_rate']()
        bits = parameters['bits']()
        radius = parameters['radius']()
        use_counts = parameters['use_counts']()
        
        train, valid, test_1, test_2 = mlp_datasets.get_ecfp_datasets(
            '../input/datasets/{}.csv'.format(dataset_name),
            bits=bits, radius=radius, use_counts=use_counts,
        )
        
        model = mlp_models.MLPModel(
            hidden_units=[num_units] * num_layers,
            dropout_rate=dropout_rate,
            loss_fn=tf.keras.losses.Huber,
            optimizer=tf.keras.optimizers.Adam,
            initial_learning_rate=learning_rate,
        )
        
        
        if test_2 is not None:
            additional_datasets = {
                'valid':  [valid['X'],  valid['y']],
                'test_1': [test_1['X'], test_1['y']],
                'test_2': [test_2['X'], test_2['y']],
            }
        else:
            additional_datasets = {
                'valid':  [valid['X'],  valid['y']],
                'test': [test_1['X'], test_1['y']],
            }
        
        model.fit(train['X'], train['y'], 
                  additional_datasets=additional_datasets,
                  batch_size=batch_size, verbose=0,
                  epochs=num_epochs)
        
        if not os.path.isdir(f'../output/learning_curves/{dataset_name}/mlp/'):
            os.makedirs(f'../output/learning_curves/{dataset_name}/mlp/')
                
        for k, v in model.learning_curves.items():
            np.save(f'../output/learning_curves/{dataset_name}/mlp/{k}_{i}.npy', 
                    np.array(list(v)))
        
        
        trues, preds = model.predict(valid['X'], valid['y'])

        error = metrics.get('mae')(trues, preds)
        
        print(error)
        
        if error < best_error:
            best_error = error
            best_params = {
                "num_epochs": num_epochs,
                "batch_size": batch_size,
                "hidden_units": [num_units] * num_layers,
                "initial_learning_rate": learning_rate,
                "dropout_rate": dropout_rate,
                "bits": bits,
                "radius": radius,
                "use_counts": use_counts,
            }
            best_weights = model.get_weights()
            
            for k, v in model.learning_curves.items():
                np.save(f'../output/learning_curves/{dataset_name}/mlp/{k}_best.npy', 
                        np.array(list(v)))

    generate_output(
        model_obj=mlp_models.MLPModel,
        model_params=best_params,
        model_weights=best_weights,
        save_path='../output/predictions/{}/{}'.format(
            dataset_name, model_name)
    )

0.7811136976877847
0.8921930249532062
0.6078309163069113
0.7071530941205145
0.796317323782505
0.6611532186850523
0.6005431714424719
0.6156541971059946
0.9457356717036319
0.6313683228615002
0.6761680451417579
0.5424267450968424
0.5323121275045932
0.5530084920540833
0.8333074294603787
0.6600142734478681
0.5801115621664584
0.5153267366458208
0.709316985790546
0.6746473009158402
{'num_epochs': 107, 'batch_size': 64, 'hidden_units': [684, 684], 'initial_learning_rate': 0.00024208352248131275, 'dropout_rate': 0.1720721051812883, 'bits': 1603, 'radius': 1, 'use_counts': False}
1.3315160651098596


In [5]:
# {'num_epochs': 163, 'batch_size': 128, 'hidden_units': [677, 677], 'initial_learning_rate': 0.0010187757243303552, 'dropout_rate': 0.21726993516627638, 'bits': 1722, 'radius': 1}